# Matrix Factorization

In [1]:
import findspark
findspark.init("/usr/spark-2.4.1")
import pyspark
from pyspark import SQLContext

pyspark.SparkContext.setSystemProperty('spark.executor.memory', '14g')
sc = pyspark.SparkContext()
sql = SQLContext(sc)

In [2]:
datafile = "data/training_sample.tsv"

df = (sql.read
    .format("csv")
    .option("header", "false")
    .option("sep", "\x01")
    .load(datafile,  inferSchema="true")
    .toDF("text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains","tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count","engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified","engaging_user_account_creation", "engaged_follows_engaging", "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"))

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer


user_indexer = StringIndexer(inputCol="engaging_user_id", outputCol="user")
tweet_indexer = StringIndexer(inputCol="tweet_id", outputCol="tweet")

pipeline = Pipeline(stages=[user_indexer, tweet_indexer])
df = pipeline.fit(df).transform(df)

In [4]:
from pyspark.sql.functions import  when, col

target_cols = ["reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]

def encode_response(x):
    return when(col(x).isNull(), float(0)).otherwise(float(1))

#def implicit_feedback(creation_time, interaction_time):
#    return when(col(interaction_time).isNull(), float(0)).otherwise(col(interaction_time)-col(creation_time))

for target_col in target_cols:
    df = df.withColumn(target_col[:-10], encode_response(target_col))


In [5]:
df = df.select("user", "tweet", "reply", "retweet", "retweet_with_comment", "like")

In [6]:
(training, test) = df.randomSplit([0.8, 0.2])

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

models = {}

maxIter=20
regParam=0.001
rank=20

for target_col in target_cols:
    target_col = target_col[:-10]
    print("Training Model for {}".format(target_col))
    models[target_col] = ALS(maxIter=maxIter, regParam=regParam, rank=rank, 
          userCol="user", itemCol="tweet", ratingCol=target_col,
          coldStartStrategy="drop", implicitPrefs=True).fit(training)
    
    # Evaluate the model by computing the RMSE on the test data
    test = models[target_col].transform(test)
    test = test.withColumnRenamed("prediction", target_col+"_pred", )
    

Training Model for reply
Training Model for retweet
Training Model for retweet_with_comment
Training Model for like


In [8]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

metrics = {}

for target_col in target_cols:
    target_col = target_col[:-10]
    predictionAndLabels = test.rdd.map(lambda r: (r[target_col+"_pred"], r[target_col]))
    metric = BinaryClassificationMetrics(predictionAndLabels)
    metrics[target_col] = metric.areaUnderPR
    print("For {}: Area under PR = {}".format(target_col, metrics[target_col]))

For reply: Area under PR = 0.00847457627118644
For retweet: Area under PR = 0.22997977540256315
For retweet_with_comment: Area under PR = 0.0021141649048625794
For like: Area under PR = 0.7354671614390181


In [9]:
a = sc.parallelize([metric.areaUnderPR])

In [10]:
a.coalesce(1).saveAsTextFile("test")

Py4JJavaError: An error occurred while calling o584.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/data/test already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	

In [ ]:
def convert_to_class(x, threshold):
    return when(col(x) <= threshold, float(0)).otherwise(float(1))

predictions = predictions.withColumn("prediction_class", convert_to_class("prediction", 0))

In [ ]:
predictions.take(10)